<a href="https://colab.research.google.com/github/wesslen/llm-excel-ingestor/blob/main/notebooks/01_excel_ingestor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import openpyxl
from pathlib import Path
import json
import re
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.worksheet import Worksheet
from typing import Dict, List, Tuple, Any

class ExcelToLLMConverter:
    def __init__(self, input_path: str, output_dir: str):
        self.input_path = Path(input_path)
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def extract_formula_metadata(self, cell: openpyxl.cell.Cell) -> Dict[str, Any]:
        """Extract metadata from a cell including its formula and dependencies."""
        metadata = {
            "address": f"{get_column_letter(cell.column)}{cell.row}",
            "value": cell.value,
            "data_type": str(cell.data_type),
            "formula": None
        }

        # Check if cell contains a formula
        if cell.data_type == 'f':
            formula = str(cell.value)
            if formula.startswith('='):
                metadata["formula"] = formula
                # Extract cell references from formula
                references = re.findall(r'[A-Z]+[0-9]+', formula)
                metadata["dependencies"] = references

        return metadata

    def process_worksheet(self, worksheet: Worksheet) -> Dict[str, Any]:
        """Process a worksheet and extract its structure and metadata."""
        sheet_data = {
            "name": worksheet.title,
            "dimensions": f"{worksheet.dimensions}",
            "cells": {},
            "formulas": {},
            "data_relationships": []
        }

        # Process each cell in the worksheet
        for row in worksheet.iter_rows():
            for cell in row:
                try:
                    if cell.value is not None:  # Only process non-empty cells
                        metadata = self.extract_formula_metadata(cell)
                        cell_address = metadata["address"]

                        # Store basic cell data
                        display_value = cell.value
                        if cell.data_type == 'f':
                            # For formula cells, try to get the calculated value
                            try:
                                display_value = cell._value
                            except:
                                display_value = "Formula result not available"

                        sheet_data["cells"][cell_address] = {
                            "value": str(display_value),  # Convert to string to handle various types
                            "type": metadata["data_type"]
                        }

                        # Store formula information if present
                        if metadata.get("formula"):
                            sheet_data["formulas"][cell_address] = {
                                "formula": metadata["formula"],
                                "dependencies": metadata.get("dependencies", [])
                            }

                            # Record data relationships
                            for dep in metadata.get("dependencies", []):
                                sheet_data["data_relationships"].append({
                                    "source": dep,
                                    "target": cell_address,
                                    "type": "formula_dependency"
                                })
                except Exception as e:
                    print(f"Error processing cell {get_column_letter(cell.column)}{cell.row}: {str(e)}")
                    continue

        return sheet_data

    def convert_to_markdown(self, sheet_data: Dict[str, Any], output_file: Path) -> None:
        """Convert sheet data to a markdown file optimized for LLM ingestion."""
        with output_file.open('w', encoding='utf-8') as f:
            # Write header
            f.write(f"# Sheet: {sheet_data['name']}\n\n")
            f.write(f"Dimensions: {sheet_data['dimensions']}\n\n")

            # Write cell values section
            f.write("## Cell Values\n\n")
            f.write("| Cell | Value | Type |\n")
            f.write("|------|--------|------|\n")
            for addr, cell_data in sheet_data["cells"].items():
                # Escape pipe characters in cell values
                safe_value = str(cell_data['value']).replace('|', '\\|')
                f.write(f"| {addr} | {safe_value} | {cell_data['type']} |\n")

            # Write formulas section
            if sheet_data["formulas"]:
                f.write("\n## Formulas\n\n")
                for addr, formula_data in sheet_data["formulas"].items():
                    f.write(f"### Cell {addr}\n")
                    f.write(f"- Formula: `{formula_data['formula']}`\n")
                    if formula_data['dependencies']:
                        f.write(f"- Dependencies: {', '.join(formula_data['dependencies'])}\n")
                    f.write("\n")

            # Write data relationships section
            if sheet_data["data_relationships"]:
                f.write("\n## Data Relationships\n\n")
                for rel in sheet_data["data_relationships"]:
                    f.write(f"- {rel['source']} → {rel['target']} ({rel['type']})\n")

    def process_workbook(self, excel_file: Path) -> None:
        """Process an entire workbook and generate output files."""
        try:
            print(f"Processing {excel_file}...")
            workbook = openpyxl.load_workbook(excel_file, data_only=False)

            # Create output directory for this workbook
            workbook_dir = self.output_dir / excel_file.stem
            workbook_dir.mkdir(exist_ok=True)

            # Process each worksheet
            for worksheet in workbook.worksheets:
                print(f"Processing worksheet: {worksheet.title}")
                sheet_data = self.process_worksheet(worksheet)

                # Save as markdown
                md_file = workbook_dir / f"{worksheet.title}.md"
                self.convert_to_markdown(sheet_data, md_file)
                print(f"Created markdown file: {md_file}")

                # Save raw data as JSON for potential other uses
                json_file = workbook_dir / f"{worksheet.title}.json"
                with json_file.open('w', encoding='utf-8') as f:
                    json.dump(sheet_data, f, indent=2)
                print(f"Created JSON file: {json_file}")

        except Exception as e:
            print(f"Error processing {excel_file}: {str(e)}")

    def convert_all(self):
        """Convert all Excel files in the input path."""
        if self.input_path.is_file():
            self.process_workbook(self.input_path)
        else:
            for excel_file in self.input_path.glob("*.xlsx"):
                self.process_workbook(excel_file)

# Example usage
if __name__ == "__main__":
    converter = ExcelToLLMConverter(
        input_path="input",
        output_dir="output"
    )
    converter.convert_all()

Processing input/balance-sheet.xlsx...
Processing worksheet: balance-sheet
Created markdown file: output/balance-sheet/balance-sheet.md
Created JSON file: output/balance-sheet/balance-sheet.json
